### KINETIC DISSOLUTION OF QUARTZ

Example taken from https://hydrochemistry.eu/exmpls/kin_qu.html

PHREEQC can model kinetic reactions of any type in a perfectly general and flexible manner.
Kinetic reaction rates are defined with keyword RATES in the form of BASIC programs. The rates are called with keyword KINETICS, which defines the time steps, the chemical formula of the reactant, and parameters that are used in the rate.
BASIC programs consist of numbered lines with statements that resemble the formula in a spreadsheet cell. Special functions give access to the chemical variables of PHREEQC. These functions are described in the PHREEQC manual in "The Basic interpreter", Table 8.

A simple example is the kinetic dissolution of quartz

```
# Kinetic dissolution of quartz
#
RATES
 Quartz
 # d qu / dt = (A0 / V) * (m / m0)^0.67 * k * (1 - SI_qu)
 # Specific rate k from Rimstidt and Barnes, 1980, GCA 44, 1683
 #  k = 10^-13.7 mol/m2/s (25 C)
 #  A0, initial surface of quartz (m2) recalc's to mol/s
 #  V, solution volume in contact with A0
 -start
  1 A0 = parm(1)
  2 V = parm(2)
  10 rate = (A0 / V) * (m/m0)^0.67 * 10^-13.7 * (1 -  SR("Quartz"))
  20 save rate * time
 -end
KINETICS
Quartz
 -formula SiO2
 -m0  158.8                        # initial moles of quartz
 -parms 23.13  0.16
 -time_step 5 year in 15           # 15 time steps
INCREMENTAL_REACTIONS true

SOLUTION 1
USER_GRAPH
 -chart_title "Quartz dissolution"
 -axis_titles Years "mmol / L"
 -axis_scale x_axis 0 5
 -initial_solutions  true
 -start
 10 graph_x total_time / 3.1536e7  # time in years on x-axis
 20 graph_y tot("Si") * 1e3        # parameter on y-axis
 -end
END
```

Dissolution of quartz,
     SiO$_2$ + 2H$_2$O ↔ H$_4$SiO$_4$
proceeds kinetically:

\begin{equation}
\frac{d[Si]}{dt}=\frac{A0}{V} \frac{m}{m_0}0.67 k (1 - SR)
\end{equation}

where $[Si]$ is the concentration of H4SiO4 (mol/L); $t$ is time (s); $A0$ is the initial surface of quartz (m2); $V$ is the solution volume ($L$); $m$ is the remaining mass of quartz (moles); $m0$ is the initial mass of quartz (moles); $k$ is the rate constant = 10$^{-13.7}$ mol/m2/s (25 C); $SR$ is the Saturation Ratio for quartz.


In [ ]:
import subprocess

# Use of the reaction command. Example of calcite dissolution in water 

input_file_content = """
# Kinetic dissolution of quartz
#
RATES
 Quartz
 # d qu / dt = (A0 / V) * (m / m0)^0.67 * k * (1 - SI_qu)
 # Specific rate k from Rimstidt and Barnes, 1980, GCA 44, 1683
 #  k = 10^-13.7 mol/m2/s (25 C)
 #  A0, initial surface of quartz (m2) recalc's to mol/s
 #  V, solution volume in contact with A0
 -start
  1 A0 = parm(1)
  2 V = parm(2)
  10 rate = (A0 / V) * (m/m0)^0.67 * 10^-13.7 * (1 -  SR("Quartz"))
  20 save rate * time
 -end
KINETICS
Quartz
 -formula SiO2
 -m0  158.8                        # initial moles of quartz
 -parms 23.13  0.16
 -time_step 5 year in 15           # 15 time steps
INCREMENTAL_REACTIONS true

SOLUTION 1
USER_GRAPH
 -chart_title "Quartz dissolution"
 -axis_titles Years "mmol / L"
 -axis_scale x_axis 0 5
 -initial_solutions  true
 -start
 10 graph_x total_time / 3.1536e7  # time in years on x-axis
 20 graph_y tot("Si") * 1e3        # parameter on y-axis
 -end
END
"""

# Save the input file
input_file_name = "phreeqc_example11.pqi"
with open(input_file_name, "w") as file:
    file.write(input_file_content)
print(f"PHREEQC input file '{input_file_name}' created successfully.")

# Step 2: Run PHREEQC using subprocess
output_file_name = "phreeqc_example11_out.txt"
database_file = "/srv/data/phreeqc-3.7.3-15968/database/phreeqc.dat"  # Update the path if necessary
phreeqc_executable = "/srv/data/phreeqc-3.7.3-15968/src/phreeqc"  # Use "phreeqc.exe" on Windows, or the full path to the executable

# Run PHREEQC
try:
    subprocess.run([phreeqc_executable, input_file_name, output_file_name, database_file], check=True)
    print(f"PHREEQC run completed. Output saved in '{output_file_name}'.")
except subprocess.CalledProcessError as e:
    print(f"PHREEQC execution failed: {e}")    

# Display the contents of the output file, ignoring problematic characters
try:
    with open(output_file_name, "r", encoding="utf-8", errors="ignore") as output_file:
        output_content = output_file.read()
    print("PHREEQC Output:\n")
    print(output_content)
except FileNotFoundError:
    print(f"Output file '{output_file_name}' not found.")